<a href="https://colab.research.google.com/github/kasettakorn/Machine_Learning/blob/master/Project_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Indoor & Outdoor Scene Recognition

# Loading Dataset from GitHub

In [0]:
!git clone https://github.com/kasettakorn/Machine_Learning.git
!pip install opencv-contrib-python          

In [0]:
import cv2
import os
import numpy as np
import pandas as pd
from scipy import interp
from itertools import cycle
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf

folders = ['Machine_Learning/Datasets/Images/bedroom',
'Machine_Learning/Datasets/Images/bookstore',
'Machine_Learning/Datasets/Images/concert_hall',
'Machine_Learning/Datasets/Images/movietheater',
'Machine_Learning/Datasets/Images/trainstation']
sess = tf.Session()
images = []
labels = []
room_label = []
def load_image_from_folder(folder):
    print(folder)
    room_label.append(folder.split(os.path.sep)[-1])
    for filename in os.listdir(folder):
        try:
            img = cv2.imread(os.path.join(folder, filename))
            img = cv2.resize(img, (300,300))
            img = image.img_to_array(img)
            label = folder.split(os.path.sep)[-1]
            labels.append(label)
            images.append(img)
           
        except:
       
            continue

for folder in folders:
    load_image_from_folder(folder)
x = np.array(images)/255
y = np.array(labels)
print(room_label)

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
y = np.array(onehot_encoded) 
print(y)

# Random sampling (train_test_split)

In [0]:
from sklearn.model_selection import train_test_split
(x_train, x_test, y_train, y_test) = train_test_split(x, y, test_size=0.15, random_state=42)

print("x_train shape = ", x_train.shape)
print("y_train shape = ",y_train.shape)
print("x_test shape = ",x_test.shape)
print("y_test shape = ",y_test.shape)


# Convolution Definition

In [0]:
#Convolution-NN
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.layers import Flatten, Dense
from keras.layers import BatchNormalization

cnn = Sequential()

cnn.add(Conv2D(16, (3, 3), input_shape=(300,300,3), activation='relu'))
cnn.add(MaxPooling2D(pool_size = (3, 3)))
cnn.add(Activation('relu'))


cnn.add(Conv2D(32, (3, 3) ,activation='relu'))
cnn.add(MaxPooling2D(pool_size = (3, 3)))


cnn.add(Conv2D(64, (3, 3) ,activation='relu'))
cnn.add(MaxPooling2D(pool_size = (3, 3)))

cnn.add(Conv2D(64, (3, 3) ,activation='relu'))
cnn.add(Conv2D(64, (3, 3) ,activation='relu'))
cnn.add(MaxPooling2D(pool_size = (3, 3)))


cnn.add(Flatten())
cnn.add(Dense(150, activation='relu'))
cnn.add(Dense(80, activation='relu'))
cnn.add(Dense(5, activation='softmax'))

cnn.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
print(cnn.summary())


history = cnn.fit(x_train, y_train, epochs=25, batch_size=32)
plt.plot(history.history['loss'])
plt.title('MLP model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [0]:
#Test Image
result = cnn.predict_classes(x_test, batch_size=32, verbose=0)


**CNN Evaluate**


In [0]:
score = cnn.evaluate(x_test, y_test, verbose=0)
print(cnn.metrics_names)
print(score)
print("Accuracy = %.2f %%" % (score[1]*100))

# ROC Curve

In [0]:
# ROC Curve 
from sklearn.metrics import roc_curve,auc
n_classes=5
pred1=cnn.predict(x_test)
room_label = list(room_label)
fpr = dict()
tpr = dict()
roc_auc = dict()
print(result[:10])
actual = []
for i in range(len(y_test)):
    for j in range(n_classes):
        if y_test[i][j] == 1:
            actual.append(j)
actual = np.array(actual)
print(np.shape(actual))
print(np.shape(pred1))

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(np.array(pd.get_dummies(actual))[:, i], np.array(pd.get_dummies(result))[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])


all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

lw=2
plt.figure(figsize=(8,5))
plt.plot(fpr["macro"], tpr["macro"],
         label='Average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='green', linestyle=':', linewidth=4)

colors = cycle(['black', 'darkorange', 'cornflowerblue', 'blue', 'yellow'])

for i, color in zip(range(n_classes), colors):
        
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(room_label[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--',color='red', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.annotate('Random Guess',(.55,.48),color='red')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Convolution Evaluation (ROC Curve) - Room Classifier')
plt.legend(loc="lower right")
plt.show()


# Test Image with Matplotlib

In [0]:
import matplotlib.pyplot as plt
import numpy as np
a = np.random.randint(1200,size=10)
test = np.concatenate(integer_encoded)

#print(a)
#print('Example training images and their labels: ') 
#print(a)

print('Predicted Class for the labels: ')

f, axarr = plt.subplots(1, 10)
f.set_size_inches(30,30)

for i in range(10):
   # print(a[i])
    print("Image Test #", i+1, "is", room_label[test[a[i]]])
    pic = images[a[i]]*255
    axarr[i].axis("off")
    axarr[i].imshow(pic)
plt.show()